# Baseline_Modeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 불러오기
data_origin = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")
data_origin

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [3]:
# 전처리를 위한 원본 데이터 복사
data = data_origin.copy()

In [4]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [5]:
downcast(data)  # 데이터의 메모리를 줄임으로써 좀 더 효과적인 처리를 가능하게 한다.

49.6% 압축됨


,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


# Pre-processing

In [6]:
# 단순 랜덤 샘플링
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(data, test_size = 0.2, random_state=42)


# 계층적 샘플링
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data['married']):
    sss_train_set = data.loc[train_index]
    sss_test_set = data.loc[test_index]

In [7]:
# train-test set split
X_train = sss_train_set.drop('married', axis=1)
y_train = sss_train_set['married']

X_test = sss_test_set.drop('married', axis=1)
y_test = sss_test_set['married']

## Pre-processing

In [8]:
# null값 확인
X_train.isnull().sum()

gender                                0
age                                  57
age_partner                          81
importance_same_religion             47
pref_of_partner_attractive           68
pref_of_partner_sincere              68
pref_of_partner_intelligence         68
pref_of_partner_funny                76
pref_of_partner_ambitious            82
pref_of_partner_shared_interests    103
eval_by_partner_attractive          164
eval_by_partner_sincere             214
eval_by_partner_intelligence        234
eval_by_partner_funny               279
eval_by_partner_ambitous            543
eval_by_partner_shared_interests    786
my_pref_attractive                   47
my_pref_sincere                      47
my_pref_intellicence                 47
my_pref_funny                        55
my_pref_ambtition                    65
my_pref_shared_interests             80
my_eval_attractive                  161
my_eval_sincere                     222
my_eval_intelligence                233


In [9]:
# 모든 변수의 null값을 각 feature의 중앙값으로 대체
median_value = X_train.iloc[:, 1:].median()
X_train.iloc[:, 1:] = X_train.iloc[:, 1:].fillna(median_value)
X_test.iloc[:, 1:] = X_test.iloc[:, 1:].fillna(median_value)

In [12]:
# null값 처리 완료
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum())

0 0


In [13]:
# 중복값 확인
X_train.duplicated().sum()

0

## Scaling

In [14]:
# 수치형 변수들 Standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train.iloc[:, 1:] = scaler.fit_transform(X_train.iloc[:, 1:])
X_test.iloc[:, 1:] = scaler.transform(X_test.iloc[:, 1:])

In [15]:
# gender One-hot encoding
X_train['gender'] = pd.get_dummies(X_train['gender'], drop_first=True)
X_test['gender'] = pd.get_dummies(X_test['gender'], drop_first=True)

In [16]:
X_train

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked
3484,True,0.463693,-0.657160,1.920094,-0.594558,0.376490,-0.790534,0.410735,0.693468,0.505295,...,-1.154213,-0.690962,-0.243517,-2.334349,-2.811928,-2.262103,-0.418214,-0.311352,-1.181458,-1.540303
4669,True,-0.384368,1.880797,1.562186,-0.194905,1.089895,0.698979,-1.220752,0.693468,-1.079912,...,-0.104115,0.488983,-0.243517,-0.757157,-1.055419,-0.275729,-0.085922,0.263009,-0.076847,-0.581876
1616,True,0.463693,-0.939155,0.846372,-0.594558,-0.336914,-0.045777,-0.405008,1.506341,0.505295,...,-0.104115,0.488983,0.417422,0.294304,-0.469917,0.220865,1.077099,0.263009,-0.076847,-0.102663
147,True,-0.667055,-0.375165,-0.943166,-1.066947,0.116811,1.037099,0.113804,1.210456,-0.431562,...,1.471032,1.078956,0.417422,1.345765,-1.055419,0.717458,-1.415088,-0.311352,1.580069,0.855764
6819,False,-0.101681,0.188826,-0.943166,1.403705,0.376490,-0.045777,-1.220752,-0.119406,-1.872515,...,-2.729361,-1.280936,-1.565395,-2.860080,-1.055419,-2.262103,-1.780609,-0.311352,-2.838373,-2.019517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,False,0.181006,-0.657160,-0.227351,-0.194905,-0.194233,-0.641583,-0.078711,0.856043,0.505295,...,-0.629164,0.488983,-0.243517,0.294304,0.115586,0.220865,-0.816963,-2.034436,-0.076847,0.376550
2714,False,-0.384368,0.188826,-0.943166,-1.194037,0.019788,-0.045777,0.818606,0.287031,1.297898,...,-1.679262,-0.100990,-0.243517,-0.231427,0.115586,0.220865,-0.152380,-0.311352,-0.076847,0.376550
2055,False,0.181006,-1.221150,-0.227351,-0.325192,0.143920,0.015293,0.477626,0.909692,-0.902369,...,-0.629164,0.488983,0.417422,-0.231427,0.115586,0.717458,-1.149255,-0.311352,0.475458,0.855764
3512,True,-0.384368,2.162792,-0.943166,-0.994210,0.376490,-0.045777,0.410735,1.506341,-0.287308,...,-0.629164,-0.690962,0.417422,-0.757157,-0.469917,-1.268916,0.246370,0.263009,-0.076847,0.376550


In [17]:
X_test

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked
6031,False,-0.667055,1.316807,0.846372,-0.994210,0.376490,2.933249,-1.220752,-0.119406,-0.287308,...,-0.104115,-0.690962,0.417422,-1.282888,1.872095,-2.262103,1.110328,0.263009,-2.286068,-2.019517
1284,False,0.746380,3.572768,-0.585258,1.403705,0.376490,-0.790534,0.410735,-1.745153,-1.079912,...,0.945983,-0.100990,0.417422,-1.808618,0.701089,0.220865,0.711578,-0.311352,0.475458,0.855764
6091,True,0.746380,0.470821,-0.585258,0.204747,1.089895,0.698979,-1.220752,-0.119406,-1.079912,...,-0.104115,-0.100990,-1.565395,-1.808618,0.115586,-0.275729,-0.152380,1.411732,-0.629152,-1.061090
3066,True,-0.101681,-0.375165,-0.943166,-0.871117,-0.556643,-0.160470,0.285110,1.381159,0.871478,...,0.420934,0.488983,0.417422,1.345765,0.701089,0.717458,0.944182,-1.460075,1.027763,1.334977
6534,False,0.181006,0.752816,1.562186,1.004053,1.803299,0.698979,-2.036495,-1.745153,-1.079912,...,0.420934,-0.100990,0.417422,0.820034,0.701089,-1.268916,-1.315401,0.837370,-0.629152,-0.102663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178,True,0.463693,-0.375165,0.846372,-0.737633,0.215261,-0.214092,-0.081974,1.015366,0.519562,...,0.945983,1.078956,0.417422,0.820034,0.115586,0.220865,-0.052693,0.837370,0.475458,0.855764
6403,True,1.311754,-0.657160,-0.943166,-0.035044,-0.051552,-0.790534,-0.405008,0.693468,0.663816,...,-0.629164,-1.280936,-1.565395,-0.757157,-1.055419,-0.275729,0.246370,-0.885713,-0.629152,-0.102663
6484,True,-0.667055,-0.375165,1.204279,-0.594558,0.376490,1.443736,0.410735,-0.932279,-0.287308,...,0.945983,-0.690962,-1.565395,0.820034,-1.055419,1.214052,1.176786,0.837370,1.027763,0.855764
2358,True,0.746380,1.034812,-0.943166,-0.674488,0.091128,-0.641583,0.084438,1.181192,0.663816,...,-0.104115,0.488983,1.078361,1.345765,1.872095,-0.772322,-0.019464,1.411732,-0.076847,-0.102663


# Model Training

In [22]:
# lightgbm 모델 학습
from lightgbm import LGBMClassifier

params = {
    'n_estimators': 300,      # 반복 수행하려는 트리의 개수 (너무 크면 오버피팅 발생)
    'max_depth': 6,             #* 트리의 최대 깊이 (너무 크면 오버피팅 발생)   # 보통 3~12(가장 민감하므로 먼저 튜닝해야할 값) # lightgbm처럼 leaf-wise로 학습하는 경우 끝까지(-1)도 괜찮다고 한다.
    'num_leaves': 32,           #* 하나의 트리가 가질 수 있는 최대 리프 수 (2번째로 민감. max_depth와 함께 튜닝)
    'learning_rate': 0.1,       # 부스팅 스탭 반복할 때 학습률(0~1) #튜닝시 0.1~0.3, 최종 모형은 0.05 이하
}

lgbm = LGBMClassifier(**params, objective='binary', metric='accuracy', force_col_wise=True, random_state=42, verbose=0)
lgbm.fit(X_train, y_train)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(force_col_wise=True, max_depth=6, metric='accuracy',
               n_estimators=300, num_leaves=32, objective='binary',
               random_state=42, verbose=0)

In [23]:
# train 및 test에 대한 성능 평가
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_train_pred = lgbm.predict(X_train)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))
print('--------'*10)

y_test_pred = lgbm.predict(X_test)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  0.9985050082224548
F1 Score:  0.995850622406639
[[5479    0]
 [  10 1200]]
--------------------------------------------------------------------------------
<<Test Set Performance>>
Accuracy:  0.8547519426180514
F1 Score:  0.5110663983903421
[[1303   67]
 [ 176  127]]
